# Project 2: ETL Challenge

### Briana Friendt, Haley Huhtala, Ian Mac Moore and Kaylene Retka

For our project we are using data from the CDC - National Health and Examination Health Survey (2013-2014). We started by exploring the data and determining which of the data sets we wanted to use. Several CSV files were available from the survey data. We selected the demographic data set and examination data set. This project will create a data set 

The columns in each data set had variable names. We started our project by creating a variable database for column names. This allowed us to determine which columns we wanted to keep in our final database. 

In this notebook, we will transform the data by combining cleaned CSV files into a final joined table in Postgres SQL DB, as a parallel effort already done using pure SQL. Note that Tables need to be created in advance, so this doesn't save that step.

### Imports

In [1]:
import pandas as pd
from sqlalchemy import create_engine

### Import CSV files and create DataFrames

In [3]:
#demographic_modified_renamed_columns.csv
#examination_modified_renamed_columns.csv

demographic_file = "./Resources/demographic_modified_final.csv"
demographic_df = pd.read_csv(demographic_file)
demographic_df.head()

,SEQN,Gender,Age,Race,US_Citizen,Years_In_US,Marital_Status,Pregnant,Num_Family,Interview_Weight,Exam_Weight,Income
0,73557,Male,69,4,1.0,NaN,4.0,Unknown,3,13281.23739,13481.04210,4.0
1,73558,Male,54,3,1.0,NaN,1.0,Unknown,4,23682.05739,24471.76963,7.0
2,73559,Male,72,3,1.0,NaN,1.0,Unknown,2,57214.80332,57193.28538,10.0
3,73560,Male,9,3,1.0,NaN,NaN,Unknown,4,55201.17859,55766.51244,9.0
4,73561,Female,73,3,1.0,NaN,1.0,Unknown,2,63709.66707,65541.87123,15.0


In [26]:
examination_file = "./Resources/examination_modified_final.csv"
examination_df = pd.read_csv(examination_file)
examination_df.head()
print(examination_df['SEQN'].dtype)
print(demographic_df['SEQN'].dtype)

int64
int64


### Connect to SQL DB

In [5]:
connection_string = "postgres:postgres@localhost:5432/HealthStudy_db"
engine = create_engine(f'postgresql://{connection_string}')

In [6]:
# Confirm tables
engine.table_names()

['leather_odor_code',
 'examination',
 'natural_gas_odor_code',
 'race_code',
 'demographics',
 'us_citizen_code',
 'years_in_us_code',
 'marital_status_code',
 'income_code',
 'health_study']

### Combine DFs and clean data
We'll drop all rows that have NAN, as there'll be sufficient entries for the purpose of this project that contain all data for all columns.

In [39]:
healthStudy_df = examination_df.merge(demographic_df,on="SEQN")
healthStudy_df

,SEQN,BP_Systolic_mmHg,BP_Diastolic_mmHg,Weight_kg,Height_cm,BMI_kg_m2,Dominate_Hand,Grip_Strength_H2_kg,Grip_Strength_H1_kg,Leather_Odor,...,Age,Race,US_Citizen,Years_In_US,Marital_Status,Pregnant,Num_Family,Interview_Weight,Exam_Weight,Income
0,73574,122.0,56.0,56.8,158.0,22.8,Right,33.8,33.0,NaN,...,33,6,1.0,4.0,1.0,No,4,14155.31300,15397.21985,8.0
1,73577,118.0,74.0,79.7,166.2,28.9,Right,32.1,36.6,NaN,...,32,1,2.0,4.0,6.0,Unknown,5,36284.65027,39735.01707,2.0
2,73581,138.0,80.0,80.9,185.0,23.6,Right,35.8,43.8,3.0,...,50,6,2.0,4.0,1.0,Unknown,3,15902.41812,16764.26067,15.0
3,73585,106.0,70.0,92.2,175.1,30.1,Right,58.0,55.4,NaN,...,28,6,1.0,6.0,1.0,Unknown,2,18063.14386,18590.96547,7.0
4,73594,118.0,66.0,62.7,159.4,24.7,Right,28.0,33.2,NaN,...,23,2,2.0,3.0,5.0,Unknown,7,38818.86718,45736.92425,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1570,83713,118.0,66.0,62.5,162.4,23.7,Right,42.7,39.1,NaN,...,34,6,1.0,4.0,1.0,Unknown,4,14919.21878,16535.23660,6.0
1571,83717,NaN,NaN,43.6,143.2,21.3,Missing,NaN,NaN,NaN,...,80,1,2.0,6.0,2.0,Unknown,7,12010.32530,12031.43972,6.0
1572,83721,108.0,70.0,79.5,176.2,25.6,Right,41.6,37.5,3.0,...,52,3,1.0,3.0,1.0,Unknown,2,67937.15346,68456.77185,15.0
1573,83726,NaN,NaN,79.0,171.7,26.8,Right,46.4,39.0,1.0,...,40,1,2.0,5.0,4.0,Unknown,2,45268.57270,49177.29100,9.0


In [40]:
#healthStudyClean_df = healthStudy_df.drop(columns="SEQN_Exam")
healthStudyClean_df = healthStudy_df.reset_index(drop=True,inplace=False)
healthStudyClean_df


,SEQN,BP_Systolic_mmHg,BP_Diastolic_mmHg,Weight_kg,Height_cm,BMI_kg_m2,Dominate_Hand,Grip_Strength_H2_kg,Grip_Strength_H1_kg,Leather_Odor,...,Age,Race,US_Citizen,Years_In_US,Marital_Status,Pregnant,Num_Family,Interview_Weight,Exam_Weight,Income
0,73574,122.0,56.0,56.8,158.0,22.8,Right,33.8,33.0,NaN,...,33,6,1.0,4.0,1.0,No,4,14155.31300,15397.21985,8.0
1,73577,118.0,74.0,79.7,166.2,28.9,Right,32.1,36.6,NaN,...,32,1,2.0,4.0,6.0,Unknown,5,36284.65027,39735.01707,2.0
2,73581,138.0,80.0,80.9,185.0,23.6,Right,35.8,43.8,3.0,...,50,6,2.0,4.0,1.0,Unknown,3,15902.41812,16764.26067,15.0
3,73585,106.0,70.0,92.2,175.1,30.1,Right,58.0,55.4,NaN,...,28,6,1.0,6.0,1.0,Unknown,2,18063.14386,18590.96547,7.0
4,73594,118.0,66.0,62.7,159.4,24.7,Right,28.0,33.2,NaN,...,23,2,2.0,3.0,5.0,Unknown,7,38818.86718,45736.92425,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1570,83713,118.0,66.0,62.5,162.4,23.7,Right,42.7,39.1,NaN,...,34,6,1.0,4.0,1.0,Unknown,4,14919.21878,16535.23660,6.0
1571,83717,NaN,NaN,43.6,143.2,21.3,Missing,NaN,NaN,NaN,...,80,1,2.0,6.0,2.0,Unknown,7,12010.32530,12031.43972,6.0
1572,83721,108.0,70.0,79.5,176.2,25.6,Right,41.6,37.5,3.0,...,52,3,1.0,3.0,1.0,Unknown,2,67937.15346,68456.77185,15.0
1573,83726,NaN,NaN,79.0,171.7,26.8,Right,46.4,39.0,1.0,...,40,1,2.0,5.0,4.0,Unknown,2,45268.57270,49177.29100,9.0


In [41]:
healthStudyClean_df.dropna(how="any", inplace=True)
healthStudyClean_df

,SEQN,BP_Systolic_mmHg,BP_Diastolic_mmHg,Weight_kg,Height_cm,BMI_kg_m2,Dominate_Hand,Grip_Strength_H2_kg,Grip_Strength_H1_kg,Leather_Odor,...,Age,Race,US_Citizen,Years_In_US,Marital_Status,Pregnant,Num_Family,Interview_Weight,Exam_Weight,Income
2,73581,138.0,80.0,80.9,185.0,23.6,Right,35.8,43.8,3.0,...,50,6,2.0,4.0,1.0,Unknown,3,15902.41812,16764.26067,15.0
5,73596,128.0,72.0,104.0,164.7,38.3,Right,28.7,30.2,3.0,...,57,6,1.0,7.0,1.0,Unknown,3,16103.98688,16152.83824,15.0
8,73614,108.0,68.0,60.9,167.9,21.6,Right,26.0,24.9,2.0,...,55,3,1.0,9.0,2.0,Unknown,2,27773.62904,28605.44669,5.0
9,73615,138.0,56.0,55.4,145.9,26.0,Right,18.1,13.2,2.0,...,65,2,1.0,8.0,2.0,Unknown,1,10950.24881,10872.08006,3.0
12,73643,134.0,64.0,73.6,168.9,25.8,Left,37.9,40.3,3.0,...,62,6,1.0,7.0,1.0,Unknown,2,14746.09819,15654.36547,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1560,83640,134.0,52.0,58.2,144.2,28.0,Right,17.1,16.1,2.0,...,79,2,1.0,5.0,2.0,Unknown,1,22015.72137,23568.44210,7.0
1565,83676,120.0,76.0,60.0,152.1,25.9,Right,25.8,18.6,4.0,...,51,1,2.0,6.0,4.0,Unknown,7,13182.96779,12942.71292,10.0
1568,83701,128.0,60.0,69.7,165.0,25.6,Right,22.9,27.8,3.0,...,45,4,1.0,5.0,1.0,Unknown,2,29231.64735,29755.94599,5.0
1572,83721,108.0,70.0,79.5,176.2,25.6,Right,41.6,37.5,3.0,...,52,3,1.0,3.0,1.0,Unknown,2,67937.15346,68456.77185,15.0


### Send DFs to SQL

In [42]:
healthStudyClean_df.to_sql(name='health_study', con=engine, if_exists='append', index=False)

In [45]:
from sqlalchemy.orm import sessionmaker
Session = sessionmaker(bind = engine)
session = Session()
result = session.query('health_study').all()

ProgrammingError: (psycopg2.errors.UndefinedColumn) column "health_study" does not exist
LINE 1: SELECT health_study
               ^

[SQL: SELECT health_study]
(Background on this error at: http://sqlalche.me/e/f405)